In [1]:
# Importing necessary library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
books = pd.read_csv('Books.csv')


C:\Users\rahul\AppData\Local\Temp\ipykernel_20720\2568570955.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


In [4]:
books = books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]

In [5]:
books.rename(columns={"Book-Title":'title',
                      'Book-Author':'author',
                     "Year-Of-Publication":'year',
                     "Publisher":"publisher",
                     "Image-URL-L":"image_url"},inplace=True)

In [7]:
users = pd.read_csv('Users.csv')

In [8]:
users.rename(columns={"User-ID":'user_id',
                      'Location':'location',
                     "Age":'age'},inplace=True)

In [10]:
ratings = pd.read_csv('Ratings.csv')

In [11]:
ratings.rename(columns={"User-ID":'user_id',
                      'Book-Rating':'rating'},inplace=True)

In [12]:
print(books.shape, users.shape, ratings.shape, sep='\n')

(271360, 6)
(278858, 3)
(1149780, 3)


In [13]:
ratings['user_id'].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
119573        1
276706        1
276697        1
276679        1
276676        1
Name: count, Length: 105283, dtype: int64

In [14]:
x = ratings['user_id'].value_counts() > 200

In [15]:
y= x[x].index

In [16]:
ratings = ratings[ratings['user_id'].isin(y)]

In [17]:
ratings_with_books = ratings.merge(books, on='ISBN')

In [18]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [19]:
number_rating.rename(columns={'rating':'num_of_rating'},inplace=True)

In [20]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [21]:
final_rating.shape

(487671, 9)

In [22]:
final_rating = final_rating[final_rating['num_of_rating'] >= 50]

In [23]:
final_rating.shape

(61853, 9)

In [24]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [25]:
final_rating.shape

(59850, 9)

In [26]:
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values= 'rating')

In [27]:
book_pivot.fillna(0, inplace=True)

In [28]:
from scipy.sparse import csr_matrix

In [29]:
book_sparse = csr_matrix(book_pivot)

In [30]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

In [31]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

In [32]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [33]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6 )

In [34]:
book_pivot.iloc[241,:]

user_id
254       9.0
2276      0.0
2766      0.0
2977      0.0
3363      0.0
         ... 
275970    9.0
277427    0.0
277478    0.0
277639    0.0
278418    0.0
Name: Harry Potter and the Sorcerer's Stone (Book 1), Length: 888, dtype: float64

In [35]:
#keeping books name
book_names = book_pivot.index

In [36]:
np.where(book_pivot.index == '4 Blondes')[0][0]

np.int64(3)

In [37]:
ids = np.where(final_rating['title'] == "Harry Potter and the Chamber of Secrets (Book 2)")[0][0]

In [38]:
final_rating.iloc[ids]['image_url']

'http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg'

In [39]:
book_name = []
for book_id in suggestion:
    book_name.append(book_pivot.index[book_id])

In [40]:
ids_index = []
for name in book_name[0]: 
    ids = np.where(final_rating['title'] == name)[0][0]
    ids_index.append(ids)

In [41]:
for idx in ids_index:
    url = final_rating.iloc[idx]['image_url']
    print(url)

http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0439139597.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0439136369.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/043936213X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0446604232.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0440115450.01.LZZZZZZZ.jpg


In [43]:
import pickle
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(book_names,open('artifacts/book_names.pkl','wb'))
pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))

In [44]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6 )
    
    for i in range(len(suggestion)):
            books = book_pivot.index[suggestion[i]]
            for j in books:
                if j == book_name:
                    print(f"You searched '{book_name}'\n")
                    print("The suggestion books are: \n")
                else:
                    print(j)

In [45]:
book_name = "Harry Potter and the Chamber of Secrets (Book 2)"
recommend_book(book_name)

You searched 'Harry Potter and the Chamber of Secrets (Book 2)'

The suggestion books are: 

Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
The Cradle Will Fall
